In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def solve_system(A, b):
        N = len(b)
        alpha = np.zeros(N-1)
        beta = np.zeros(N-1)
        beta[0] = b[0]/A[0,0]
        alpha[0] = -A[0,1]/A[0,0]
        
        for i in range (1, N-1):
            alpha[i] = -A[i,i+1]/(A[i,i-1]*alpha[i-1] + A[i,i])
            beta[i] = (b[i] - A[i,i-1]*beta[i-1]) /(A[i,i-1]*alpha[i-1] + A[i,i])
            
        y = np.zeros(N)
        y[N-1] = (b[N-1] - A[N-1,N-2]*beta[N-2]) / (A[N-1,N-2]*alpha[N-2] + A[N-1,N-1])
        
        for i in range (N-2, -1, -1):
            y[i] = alpha[i] * y[i+1] + beta[i]
            
        return y

In [22]:
def solve_by_center_diffs(x, u_0, u_1, a, b, f, eps):
    N = len(x)
    h = 1/(N-1)
    A = np.zeros((N,N))
    r = np.zeros(N)
    A[0,0] = 1
    r[0] = u_0
    A[N-1,N-1] = 1
    r[N-1] = u_1
    for n in range(1, N-1):
        A[n,n-1] = eps/h**2 - a(x[n])/(2*h)
        A[n,n] = -2*eps/h**2 - b(x[n])
        A[n,n+1] = eps/h**2 + a(x[n])/(2*h)
        r[n] = f(x[n])
    return solve_system(A,r)

def solve_by_directed_diffs(x, u_0, u_1, a, b, f, eps):
    N = len(x)
    h = 1/(N-1)
    A = np.zeros((N,N))
    r = np.zeros(N)
    A[0,0] = 1
    r[0] = u_0
    A[N-1,N-1] = 1
    r[N-1] = u_1
    for n in range(1, N-1):
        A[n,n-1] = eps/h**2
        A[n,n] = -2*eps/h**2 - a(x[n])/h - b(x[n])
        A[n,n+1] = eps/h**2 + a(x[n])/h
        r[n] = f(x[n])
    return solve_system(A,r)

def solve_by_ilyin_scheme(x, u_0, u_1, a, b, f, eps):
    N = len(x)
    h = 1/(N-1)
    A = np.zeros((N,N))
    r = np.zeros(N)
    A[0,0] = 1
    r[0] = u_0
    A[N-1,N-1] = 1
    r[N-1] = u_1
    for n in range(1, N-1):
        eps_cur = a(x[n])*h/2 * 1/np.tanh(a(x[n]*h/(2*eps)))
        A[n,n-1] = eps/h**2 - a(x[n])/(2*h)
        A[n,n] = -2*eps/h**2 - b(x[n])
        A[n,n+1] = eps/h**2 + a(x[n])/(2*h)
        r[n] = f(x[n])
    return solve_system(A,r)

def solve_on_uneven_grid(x, u_0, u_1, a, b, f, eps):
    N = len(x)
    h = 1/(N-1)
    A = np.zeros((N,N))
    r = np.zeros(N)
    A[0,0] = 1
    r[0] = u_0
    A[N-1,N-1] = 1
    r[N-1] = u_1
    for n in range(1, N-1):
        h_n = x[n] - x[n-1]
        h_nn = x[n+1] - x[n]
        A[n,n-1] = 2*eps/((h_n+h_nn)*h_n)
        A[n,n] = -(2*eps/(h_n + h_nn)*(1/h_n + 1/h_nn) + a(x[n])/h_nn + b(x[n]))
        A[n,n+1] = 2*eps/((h_n+h_nn)*h_nn) + a(x[n])/h_nn
        r[n] = f(x[n])
    return solve_system(A,r)

def grid_x(a,b,n):
    dots=[]
    for i in range(n+1):
        dots.append(a+(b-a)*i/n)
    return np.array(dots)

def min_(a,b):
    if a > b:
        return b
    else:
        return a

def shishkin_grid(a, b, N, eps):
    sigma = min_(2*eps*np.log(N),0.5)
    res = np.zeros(N+1)
    res[0] = a
    h = 2*sigma/N
    for i in range(1,int(N/2) + 1):
        res[i] = res[i-1] + h
    H = 2*(b-a-sigma)/N
    for i in range(int(N/2) + 1, N+1):
        res[i] = res[i-1] + H
    return res

def bahvalov_grid(N, eps, k=2):
    sigma = min_(0.5, - k * eps * np.log(eps))
    if sigma > 1 / np.exp(1):
        sigma = 1/2
    if sigma >= 1/2:
        return uniform_grid(0, 1, N)
    res = []
    for i in range(int(N/2) + 1):
        res.append(- k * eps * np.log(1 - 2*(1 - eps) * i / N))
    for i in range(int(N/2) + 1, N+1):
        res.append(sigma + (2*i/N-1)*(1-sigma) )
    return np.array(res)

def norm(f_1, f_2):
    return np.max(np.abs(f_1 - f_2))

In [12]:
N = [32, 64, 128, 256, 512, 1024, 2048]
eps = [1/32, 1/64, 1/128, 1/256, 1/512, 1/1024, 1/2048]

res_table = np.zeros((len(eps), len(N)))

for i in range(len(eps)):
    for j in range(len(N)):
        x = grid_x(0,1,N[j])
        u_approximate_h = solve_by_center_diffs(x= x, u_0 = 2, u_1 = np.exp(-1/eps[i]) + 1/2, a = lambda x: 1 + x*0, \
                                              b = lambda x: 0*x, f = lambda x: 2*eps[i]/(x+1)**3-1/(1+x)**2, eps = eps[i])
        u_accuracy = lambda x: np.exp(-x/eps[i]) + 1/(x+1)
        u_accuracy_h = u_accuracy(x)
        error = norm(u_accuracy_h, u_approximate_h)
        res_table[i,j] = error
        print("{:.2e}".format(error), end=' ')
    print()
    
orders = np.zeros((len(eps), len(N)-1))
for j in range(len(N)-1):
    orders[:,j] = np.log2(res_table[:,j] / res_table[:,j+1] )

print("ORDERS:")
for i in range(len(eps)):
    for j in range(len(N)-1):
        print("{:.2e}".format(orders[i,j]),end=' ')
    print()

3.47e-02 7.92e-03 1.94e-03 4.82e-04 1.20e-04 3.01e-05 7.52e-06 
1.36e-01 3.46e-02 7.89e-03 1.93e-03 4.80e-04 1.20e-04 3.00e-05 
3.52e-01 1.35e-01 3.46e-02 7.88e-03 1.93e-03 4.80e-04 1.20e-04 
6.01e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 1.93e-03 4.80e-04 
7.79e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 1.93e-03 
9.18e-01 7.78e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 
1.24e+00 8.83e-01 7.78e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 
ORDERS:
2.13e+00 2.03e+00 2.01e+00 2.00e+00 2.00e+00 2.00e+00 
1.97e+00 2.13e+00 2.03e+00 2.01e+00 2.00e+00 2.00e+00 
1.38e+00 1.97e+00 2.13e+00 2.03e+00 2.01e+00 2.00e+00 
7.72e-01 1.38e+00 1.97e+00 2.13e+00 2.03e+00 2.01e+00 
3.75e-01 7.72e-01 1.38e+00 1.97e+00 2.13e+00 2.03e+00 
2.39e-01 3.74e-01 7.72e-01 1.38e+00 1.97e+00 2.13e+00 
4.94e-01 1.83e-01 3.74e-01 7.72e-01 1.38e+00 1.97e+00 


In [13]:
res_table = np.zeros((len(eps), len(N)))

for i in range(len(eps)):
    for j in range(len(N)):
        x = grid_x(0,1,N[j])
        u_approximate_h = solve_by_directed_diffs(x= x, u_0 = 2, u_1 = np.exp(-1/eps[i]) + 1/2, a = lambda x: 1 + x*0, \
                                              b = lambda x: 0*x, f = lambda x: 2*eps[i]/(x+1)**3-1/(1+x)**2, eps = eps[i])
        u_accuracy = lambda x: np.exp(-x/eps[i]) + 1/(x+1)
        u_accuracy_h = u_accuracy(x)
        error = norm(u_accuracy_h, u_approximate_h)
        res_table[i,j] = error
        print("{:.2e}".format(error), end=' ')
    print()
    
orders = np.zeros((len(eps), len(N)-1))
for j in range(len(N)-1):
    orders[:,j] = np.log2(res_table[:,j] / res_table[:,j+1] )

print("ORDERS:")
for i in range(len(eps)):
    for j in range(len(N)-1):
        print("{:.2e}".format(orders[i,j]),end=' ')
    print()

1.38e-01 7.96e-02 4.33e-02 2.27e-02 1.16e-02 5.89e-03 2.96e-03 
2.05e-01 1.35e-01 7.81e-02 4.26e-02 2.23e-02 1.14e-02 5.79e-03 
1.90e-01 2.02e-01 1.34e-01 7.74e-02 4.21e-02 2.21e-02 1.13e-02 
1.20e-01 1.86e-01 2.00e-01 1.33e-01 7.70e-02 4.19e-02 2.20e-02 
6.91e-02 1.16e-01 1.84e-01 1.99e-01 1.32e-01 7.68e-02 4.18e-02 
4.09e-02 6.41e-02 1.13e-01 1.83e-01 1.98e-01 1.32e-01 7.67e-02 
2.61e-02 3.58e-02 6.15e-02 1.12e-01 1.82e-01 1.98e-01 1.32e-01 
ORDERS:
7.89e-01 8.77e-01 9.33e-01 9.64e-01 9.82e-01 9.91e-01 
6.05e-01 7.88e-01 8.77e-01 9.32e-01 9.65e-01 9.82e-01 
-8.36e-02 5.95e-01 7.88e-01 8.76e-01 9.32e-01 9.64e-01 
-6.28e-01 -1.03e-01 5.90e-01 7.87e-01 8.76e-01 9.32e-01 
-7.45e-01 -6.68e-01 -1.13e-01 5.87e-01 7.87e-01 8.76e-01 
-6.50e-01 -8.21e-01 -6.90e-01 -1.18e-01 5.85e-01 7.87e-01 
-4.54e-01 -7.82e-01 -8.65e-01 -7.02e-01 -1.21e-01 5.84e-01 


In [14]:
res_table = np.zeros((len(eps), len(N)))

for i in range(len(eps)):
    for j in range(len(N)):
        x = grid_x(0,1,N[j])
        u_approximate_h = solve_by_ilyin_scheme(x= x, u_0 = 2, u_1 = np.exp(-1/eps[i]) + 1/2, a = lambda x: 1 + x*0, \
                                              b = lambda x: 0*x, f = lambda x: 2*eps[i]/(x+1)**3-1/(1+x)**2, eps = eps[i])
        u_accuracy = lambda x: np.exp(-x/eps[i]) + 1/(x+1)
        u_accuracy_h = u_accuracy(x)
        error = norm(u_accuracy_h, u_approximate_h)
        res_table[i,j] = error
        print("{:.2e}".format(error), end=' ')
    print()
    
orders = np.zeros((len(eps), len(N)-1))
for j in range(len(N)-1):
    orders[:,j] = np.log2(res_table[:,j] / res_table[:,j+1] )

print("ORDERS:")
for i in range(len(eps)):
    for j in range(len(N)-1):
        print("{:.2e}".format(orders[i,j]),end=' ')
    print()

3.47e-02 7.92e-03 1.94e-03 4.82e-04 1.20e-04 3.01e-05 7.52e-06 
1.36e-01 3.46e-02 7.89e-03 1.93e-03 4.80e-04 1.20e-04 3.00e-05 
3.52e-01 1.35e-01 3.46e-02 7.88e-03 1.93e-03 4.80e-04 1.20e-04 
6.01e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 1.93e-03 4.80e-04 
7.79e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 1.93e-03 
9.18e-01 7.78e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 7.88e-03 
1.24e+00 8.83e-01 7.78e-01 6.00e-01 3.52e-01 1.35e-01 3.45e-02 
ORDERS:
2.13e+00 2.03e+00 2.01e+00 2.00e+00 2.00e+00 2.00e+00 
1.97e+00 2.13e+00 2.03e+00 2.01e+00 2.00e+00 2.00e+00 
1.38e+00 1.97e+00 2.13e+00 2.03e+00 2.01e+00 2.00e+00 
7.72e-01 1.38e+00 1.97e+00 2.13e+00 2.03e+00 2.01e+00 
3.75e-01 7.72e-01 1.38e+00 1.97e+00 2.13e+00 2.03e+00 
2.39e-01 3.74e-01 7.72e-01 1.38e+00 1.97e+00 2.13e+00 
4.94e-01 1.83e-01 3.74e-01 7.72e-01 1.38e+00 1.97e+00 


In [15]:
res_table = np.zeros((len(eps), len(N)))

for i in range(len(eps)):
    for j in range(len(N)):
        x = shishkin_grid(0,1,N[j],eps[i])
        u_approximate_h = solve_on_uneven_grid(x= x, u_0 = 2, u_1 = np.exp(-1/eps[i]) + 1/2, a = lambda x: 1 + x*0, \
                                              b = lambda x: 0*x, f = lambda x: 2*eps[i]/(x+1)**3-1/(1+x)**2, eps = eps[i])
        u_accuracy = lambda x: np.exp(-x/eps[i]) + 1/(x+1)
        u_accuracy_h = u_accuracy(x)
        error = norm(u_accuracy_h, u_approximate_h)
        res_table[i,j] = error
        print("{:.2e}".format(error), end=' ')
    print()
    
orders = np.zeros((len(eps), len(N)-1))
for j in range(len(N)-1):
    orders[:,j] = np.log2(res_table[:,j] / res_table[:,j+1] )

print("ORDERS:")
for i in range(len(eps)):
    for j in range(len(N)-1):
        print("{:.2e}".format(orders[i,j]),end=' ')
    print()

7.56e-02 4.67e-02 2.79e-02 1.62e-02 9.20e-03 5.13e-03 2.83e-03 
7.88e-02 4.80e-02 2.85e-02 1.65e-02 9.30e-03 5.17e-03 2.84e-03 
8.13e-02 4.91e-02 2.91e-02 1.68e-02 9.45e-03 5.25e-03 2.88e-03 
8.30e-02 4.99e-02 2.95e-02 1.70e-02 9.57e-03 5.31e-03 2.91e-03 
8.39e-02 5.03e-02 2.98e-02 1.71e-02 9.64e-03 5.35e-03 2.93e-03 
8.44e-02 5.06e-02 2.99e-02 1.72e-02 9.68e-03 5.37e-03 2.94e-03 
8.47e-02 5.07e-02 3.00e-02 1.72e-02 9.71e-03 5.38e-03 2.95e-03 
ORDERS:
6.93e-01 7.42e-01 7.86e-01 8.18e-01 8.41e-01 8.59e-01 
7.15e-01 7.51e-01 7.93e-01 8.24e-01 8.47e-01 8.64e-01 
7.28e-01 7.55e-01 7.96e-01 8.26e-01 8.49e-01 8.66e-01 
7.35e-01 7.57e-01 7.98e-01 8.26e-01 8.50e-01 8.67e-01 
7.38e-01 7.58e-01 7.98e-01 8.27e-01 8.50e-01 8.67e-01 
7.38e-01 7.59e-01 7.98e-01 8.28e-01 8.50e-01 8.67e-01 
7.39e-01 7.60e-01 7.98e-01 8.28e-01 8.51e-01 8.67e-01 


In [23]:
res_table = np.zeros((len(eps), len(N)))

for i in range(len(eps)):
    for j in range(len(N)):
        x = bahvalov_grid(N[j],eps[i])
        u_approximate_h = solve_on_uneven_grid(x= x, u_0 = 2, u_1 = np.exp(-1/eps[i]) + 1/2, a = lambda x: 1 + x*0, \
                                              b = lambda x: 0*x, f = lambda x: 2*eps[i]/(x+1)**3-1/(1+x)**2, eps = eps[i])
        u_accuracy = lambda x: np.exp(-x/eps[i]) + 1/(x+1)
        u_accuracy_h = u_accuracy(x)
        error = norm(u_accuracy_h, u_approximate_h)
        res_table[i,j] = error
        print("{:.2e}".format(error), end=' ')
    print()
    
orders = np.zeros((len(eps), len(N)-1))
for j in range(len(N)-1):
    orders[:,j] = np.log2(res_table[:,j] / res_table[:,j+1] )

print("ORDERS:")
for i in range(len(eps)):
    for j in range(len(N)-1):
        print("{:.2e}".format(orders[i,j]),end=' ')
    print()

6.52e-02 3.37e-02 1.71e-02 8.62e-03 4.33e-03 2.17e-03 1.09e-03 
7.03e-02 3.60e-02 1.82e-02 9.15e-03 4.58e-03 2.29e-03 1.15e-03 
7.42e-02 3.77e-02 1.90e-02 9.51e-03 4.76e-03 2.38e-03 1.19e-03 
7.70e-02 3.90e-02 1.95e-02 9.76e-03 4.88e-03 2.44e-03 1.22e-03 
7.91e-02 3.99e-02 1.99e-02 9.93e-03 4.95e-03 2.47e-03 1.24e-03 
8.04e-02 4.05e-02 2.02e-02 1.00e-02 5.00e-03 2.50e-03 1.25e-03 
8.13e-02 4.09e-02 2.03e-02 1.01e-02 5.03e-03 2.51e-03 1.25e-03 
ORDERS:
9.55e-01 9.77e-01 9.89e-01 9.94e-01 9.97e-01 9.98e-01 
9.65e-01 9.84e-01 9.93e-01 9.97e-01 9.98e-01 9.99e-01 
9.74e-01 9.91e-01 9.97e-01 9.99e-01 9.99e-01 1.00e+00 
9.82e-01 9.97e-01 1.00e+00 1.00e+00 1.00e+00 1.00e+00 
9.87e-01 1.00e+00 1.00e+00 1.00e+00 1.00e+00 1.00e+00 
9.90e-01 1.01e+00 1.01e+00 1.00e+00 1.00e+00 1.00e+00 
9.91e-01 1.01e+00 1.01e+00 1.01e+00 1.00e+00 1.00e+00 
